- Create Lakebase Database Instance 
- Create a synced table
- Create a lakebase native table

In [0]:
%pip install --upgrade databricks-sdk

In [0]:
dbutils.library.restartPython()

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance, SyncedTableSpec, SyncedTableSchedulingPolicy, SyncedDatabaseTable

In [0]:
lakebase_instance_name = 'zg-mfg-lakebase-demo' # name of the database instance
cu = 'CU_1' # size of the database instance
node_count = 2 # number of database nodes, need at least 2 for failover

catalog_name = 'zg' # UC catalog name
pg_db_name = 'pg_mfg_db' # Lakebase database name (similar to UC catalog level)
schema_name = 'mfg_lakebase_demo' # name of the UC schema and postgres schema

routes_destination_table_name = f"{catalog_name}.{schema_name}.recommended_routes_synced_table"
routes_source_table_full_name = f"{catalog_name}.{schema_name}.recommended_routes"
routes_primary_key_columns = ["part_id"]

parts_backlog_destination_table_name = f"{catalog_name}.{schema_name}.part_backlog_synced_table"
parts_backlog_source_table_full_name = f"{catalog_name}.{schema_name}.part_backlog"
parts_backlog_primary_key_columns = ["part_id"]

pg_table_name = 'assignment_overrides' # native postgres table name

In [0]:
# Initialize the workspace client 
w = WorkspaceClient()

In [0]:
# create the database instance 
database_instance = DatabaseInstance(
  name=lakebase_instance_name,
  capacity=cu,
  node_count=node_count
)

w.database.create_database_instance(
  database_instance = database_instance
)

In [0]:
# wait for the database instance to become available before creating the synced table
w.database.wait_get_database_instance_database_available(name=lakebase_instance_name)

In [0]:
# create a synced table for the recommended routes
table_spec = SyncedTableSpec(
  source_table_full_name=routes_source_table_full_name,
  primary_key_columns=routes_primary_key_columns,
  scheduling_policy=SyncedTableSchedulingPolicy.TRIGGERED, 
)

synced_table = SyncedDatabaseTable(
  name=routes_destination_table_name,
  database_instance_name=lakebase_instance_name,
  logical_database_name=pg_db_name,
  spec=table_spec
)

w.database.create_synced_database_table(synced_table=synced_table)

In [0]:
routes_destination_table_name_info = w.database.get_synced_database_table(name=routes_destination_table_name)
pipeline_id = routes_destination_table_name_info.data_synchronization_status.pipeline_id

In [0]:
# create a synced table for the part backlog using the same DLT pipeline
table_spec = SyncedTableSpec(
  existing_pipeline_id=pipeline_id,
  source_table_full_name=parts_backlog_source_table_full_name,
  primary_key_columns=parts_backlog_primary_key_columns,
  scheduling_policy=SyncedTableSchedulingPolicy.TRIGGERED, 
)

synced_table = SyncedDatabaseTable(
  name=parts_backlog_destination_table_name,
  database_instance_name=lakebase_instance_name,
  logical_database_name=pg_db_name,
  spec=table_spec
)

w.database.create_synced_database_table(synced_table=synced_table)

In [0]:
# run the pipeline to populate the part_log table
run = w.pipelines.start_update(
    pipeline_id=pipeline_id,
    full_refresh=False
)

In [0]:
# create a native lakebase (postgres) table
import psycopg2
import uuid

user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

instance = w.database.get_database_instance(name=lakebase_instance_name)
cred = w.database.generate_database_credential(request_id=str(uuid.uuid4()), instance_names=[lakebase_instance_name])

# Connection parameters
conn = psycopg2.connect(
    host = instance.read_write_dns,
    dbname = pg_db_name,
    user = user,
    password = cred.token,
    sslmode = "require"
)

# Execute query
with conn.cursor() as cur:
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {schema_name}.{pg_table_name} (
      part_id VARCHAR(255),
      assigned_machine_id VARCHAR(255),
      assigned_by VARCHAR(255),
      assigned_at TIMESTAMP,
      notes TEXT
    );
    """)

    cur.execute(f"""
    SELECT EXISTS (
        SELECT 1 FROM information_schema.tables
        WHERE table_schema = '{schema_name}' AND table_name = '{pg_table_name}'
    );
    """)
    created = cur.fetchone()[0]
    if created:
        print(f"Table {schema_name}.{pg_table_name} has been created or already exists.")
    else:
        print("Table creation failed or is not visible.")
conn.commit()
conn.close()